In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import keras.backend.tensorflow_backend as K

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
#dnn_input_data 불러오기 및 인덱스값 'Author'로 변환
dnn_input_data = pd.read_csv('DNN_input_data.csv')
del dnn_input_data['Unnamed: 0']
dnn_input_data = dnn_input_data.set_index('Author')
dnn_input_data.drop(['HotelInfo.Name','HotelInfo.HotelID','ReviewID','Business service (e.g., internet access)','Check in / front desk','Title','Content',
               'AuthorLocation','Date'],axis='columns',inplace=True)

# 함수

## UPC 생성

### 유저리스트 생성

In [3]:
def user_list_fu(dnn_input_data):
    user_list = dnn_input_data.index
    user_list = user_list.unique()
    return user_list

### DNN

In [4]:
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Input
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
UPC_list = [] #학습을 통해 나온 가중치 리스트 저장 공간


In [5]:
def train_test_data(one_user):    
    x_train, x_test,y_train, y_test = train_test_split(one_user.iloc[:,0:6],one_user.iloc[:,6], random_state=66, test_size=0.3)
    return x_train,x_test,y_train,y_test

In [6]:
def dnn_model():
    #2. 모델 구성
    input1 = Input(shape=(6,))
    #dense1 = Dense(5, activation='relu')(input1)
    #dense2 = Dense(3)(dense1)
    output = Dense(1, activation='sigmoid')(input1)
    model = Model(inputs=input1, outputs = output)
    
    model.compile(optimizer='adam', loss='MSE',metrics=['accuracy'])

    return model

In [7]:
def train_test_data(one_user):    
    x_train, x_test,y_train, y_test = train_test_split(one_user.iloc[:,0:6],one_user.iloc[:,6], random_state=66, test_size=0.3)
    return x_train,x_test,y_train,y_test

In [8]:
def model_fit(model, x_train, y_train, x_test, y_test):
    # 3. 훈련
    model.fit(x_train,y_train/5, epochs=100, batch_size=1, validation_data=(x_test,y_test/5))
    
    # 4. 평가 예측
    loss, mse = model.evaluate(x_test,y_test/5,batch_size=1)
    print('acc:',mse)
    return model

In [9]:
#def predict_y(model,x_test):
# 5. 결과 예측 및 가중치 추출
#    y_predict = model.predict(x_test)
#    return y_predict

### 가중치 추출

In [10]:
def weights_list(model):
    weights = model.layers[1].get_weights()
    weights = MinMaxScaler().fit_transform(weights[0])
    weights = np.array(weights)
    weights= np.squeeze(weights,axis=1)
    UPC_list=[]
    UPC_list.append(weights)
    return UPC_list

### DNN_main 함수

In [11]:
def dnn_main(one_user):
    x_train,x_test,y_train,y_test = train_test_data(one_user)
    
    model = dnn_model()
    model = model_fit(model, x_train, y_train, x_test, y_test)
    #pred_y = predict_y()
    
    UPC_list=weights_list(model)
    
    return UPC_list

### 데이터프레임화

In [12]:
def UPC_dataframe(UPC_list,user_list):   
    UPC_data = pd.DataFrame(index=range(0,0),columns=['Author','Service','Cleanliness','Rooms','Value','Sleep Quality','Location'])
    UPC_data = UPC_data.set_index('Author')

    for i in range(len(UPC_list)):
        UPC_data = UPC_data.append(pd.Series(a[i],name = user_list[i],index=UPC_data.columns))

    return UPC_data
    

### 10~50개 리뷰를 작성한 사람으로 데이터 전처리

In [13]:
def len_pre(data_over):
    data_count = data_over['Author'].value_counts()
    tmp = pd.DataFrame(data_count.index, columns=['Author'])
    tmp['count'] = data_count.values
    data_count = tmp


    np_data = np.array(data_count)
    np_data=np_data[np.where(np_data[:,1]>=10)]
    np_data=np_data[np.where(np_data[:,1]<=50)]

    np_data = np.delete(np_data,(1),axis=1)
    df1 = pd.DataFrame(columns=['name'])
    name = np_data.reshape(-1,)
    df1['name']= name
    data_over= data_over[data_over['Author'].isin(df1['name'])]
    data_over = data_over.reset_index(drop=True)
    
    return data_over

## 행렬곱 연산

In [14]:
def matrix_multiplication(IPC,UPC):
    #호텔, 저자 이름 추출
    hotel_name = IPC['HotelInfo.HotelID']
    test_IPC = drop(['HotelInfo.HotelID'],axis=1)

    Author_name = UPC['Author']
    test_UPC = UPC.drop(['Author'],axis=1)
    ##################################################################

    trans_UPC=np.transpose(test_UPC)

    predict_matrix = np.dot(test_IPC,trans_UPC)
    predict_matrix = pd.DataFrame(predict_matrix)

    predict_matrix.columns=Author_name
    predict_matrix.index = hotel_name
    
    return predict_matrix


    predict_matrix = matrix_multiplication(data_IPC,data_UPC)
    
    return predict_matrix

## main()

In [15]:
def main(dnn_input_data):
    with K.tf.device('/gpu:0'):
        user_list = user_list_fu(dnn_input_data)
        for i in range(len(user_list)):#유저 수만큼 반복
            one_user= dnn_input_data.loc[user_list[i]]#유저 1명에 대한 리뷰
            UPC_list = dnn_main(one_user)#main()함수로 DNN학습 후 유저별 가중치를 list에 저장

        UPC = UPC_dataframe(UPC_list,user_list)#UPC_list를 데이터 프레임화
        
        predict_matrix = matrix_multiplication(IPC,UPC):
         
        return predict_matrix

# 실행

In [16]:
predict_matrix = main(dnn_input_data)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Train on 4 samples, validate on 2 samples
Epoch 1/100
4/4 [==============================] - 0s 44ms/sample - loss: 0.9995 - acc: 0.0000e+00 - val_loss: 0.9988 - val_acc: 0.0000e+00
Epoch 2/100
4/4 [==============================] - 0s 2ms/sample - loss: 0.9995 - acc: 0.0000e+00 - val_loss: 0.9988 - val_acc: 0.0000e+00
Epoch 3/100
4/4 [==============================] - 0s 2ms/sample - loss: 0.9994 - acc: 0.0000e+00 - val_loss: 0.9987 - val_acc: 0.0000e+00
Epoch 4/100
4/4 [==============================] - 0s 2ms/sample - loss: 0.9994 - acc: 0.0000e+00 - val_loss: 0.9986 - val_acc: 0.0000e+00
Epoch 5/100
4/4 [==============================] - 0s 2ms/sample - loss: 0.9994 - acc: 0.0000e+00 - val_loss: 0.9986 - val_acc: 0.0000e+00
Epoch 6/100
4/4 [==============================] - 0s 2ms/sample - loss: 0.9993 - acc: 0.0000e+00 - val_loss: 0.9985 - val_acc: 0.0000e+00
Epoch

Epoch 57/100
4/4 [==============================] - 0s 2ms/sample - loss: 0.9350 - acc: 0.0000e+00 - val_loss: 0.8734 - val_acc: 0.0000e+00
Epoch 58/100
4/4 [==============================] - 0s 2ms/sample - loss: 0.9279 - acc: 0.0000e+00 - val_loss: 0.8538 - val_acc: 0.0000e+00
Epoch 59/100
4/4 [==============================] - 0s 1ms/sample - loss: 0.9158 - acc: 0.0000e+00 - val_loss: 0.8310 - val_acc: 0.0000e+00
Epoch 60/100
4/4 [==============================] - 0s 2ms/sample - loss: 0.8970 - acc: 0.0000e+00 - val_loss: 0.8067 - val_acc: 0.0000e+00
Epoch 61/100
4/4 [==============================] - 0s 2ms/sample - loss: 0.8826 - acc: 0.0000e+00 - val_loss: 0.7766 - val_acc: 0.0000e+00
Epoch 62/100
4/4 [==============================] - 0s 2ms/sample - loss: 0.8643 - acc: 0.0000e+00 - val_loss: 0.7414 - val_acc: 0.0000e+00
Epoch 63/100
4/4 [==============================] - 0s 2ms/sample - loss: 0.8375 - acc: 0.0000e+00 - val_loss: 0.7022 - val_acc: 0.0000e+00
Epoch 64/100
4/4 [==

9/9 [==============================] - 0s 1ms/sample - loss: 0.0390 - acc: 0.4444 - val_loss: 0.0811 - val_acc: 0.7500
Epoch 77/100
9/9 [==============================] - 0s 1ms/sample - loss: 0.0387 - acc: 0.4444 - val_loss: 0.0810 - val_acc: 0.7500
Epoch 78/100
9/9 [==============================] - 0s 1ms/sample - loss: 0.0384 - acc: 0.4444 - val_loss: 0.0808 - val_acc: 0.7500
Epoch 79/100
9/9 [==============================] - 0s 1ms/sample - loss: 0.0383 - acc: 0.4444 - val_loss: 0.0810 - val_acc: 0.7500
Epoch 80/100
9/9 [==============================] - 0s 1ms/sample - loss: 0.0379 - acc: 0.4444 - val_loss: 0.0810 - val_acc: 0.7500
Epoch 81/100
9/9 [==============================] - 0s 1ms/sample - loss: 0.0377 - acc: 0.4444 - val_loss: 0.0806 - val_acc: 0.7500
Epoch 82/100
9/9 [==============================] - 0s 1ms/sample - loss: 0.0373 - acc: 0.4444 - val_loss: 0.0806 - val_acc: 0.7500
Epoch 83/100
9/9 [==============================] - 0s 1ms/sample - loss: 0.0371 - acc: 0

KeyboardInterrupt: 